In [1]:
import pandas as pd
import numpy as np

In [8]:
def clean_data(df):
    
    # Cleans the input DataFrame by handling missing and invalid data.
    # Returns a cleaned DataFrame and a log of changes.
    
    log = []

    # Step 1: Track original null values
    missing_before = df.isnull().sum()
    
    # Step 2: Handle missing values
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in [np.float64, np.int64]:
                fill_value = df[col].median()
                df[col] = df[col].fillna(fill_value)
                log.append(f"Filled {df[col].isnull().sum()} missing values in '{col}' with median: {fill_value}")
            elif df[col].dtype == object:
                fill_value = df[col].mode()[0] if not df[col].mode().empty else "Unknown"
                df[col] = df[col].fillna(fill_value)
                log.append(f"Filled {df[col].isnull().sum()} missing values in '{col}' with mode: '{fill_value}'")
    
    # Step 3: Convert columns to appropriate data types (basic fix for invalid entries)
    for col in df.columns:
        if df[col].dtype == object:
            # Try converting to numeric if it seems numeric
            try:
                df[col] = pd.to_numeric(df[col])
                log.append(f"Converted column '{col}' to numeric type.")
            except:
                continue  # Leave as is if conversion fails

    # Step 4: Remove or fix obviously invalid data (e.g., negative ages or zero prices)
    for col in df.select_dtypes(include=[np.number]).columns:
        if (df[col] < 0).any():
            num_negatives = (df[col] < 0).sum()
            df.loc[df[col] < 0, col] = np.nan
            df[col] = df[col].fillna(df[col].median())
            log.append(f"Replaced {num_negatives} negative values in '{col}' with median.")
    
    # Step 5: Final missing check
    missing_after = df.isnull().sum()
    for col in df.columns:
        if missing_after[col] != missing_before[col]:
            log.append(f"Missing values in '{col}' changed from {missing_before[col]} to {missing_after[col]}")
    
    return df, log


In [13]:
df=pd.read_csv("student_records_50_rows.csv")
cleaned_df, change_log = clean_data(df)
for entry in change_log:
    print(entry)


Filled 0 missing values in 'School ID' with median: 105.0
Filled 0 missing values in 'Address' with mode: '101 Elm St'
Filled 0 missing values in 'City' with mode: 'Miami'
Filled 0 missing values in 'Marks' with median: 78.19999999999999
Missing values in 'School ID' changed from 3 to 0
Missing values in 'Address' changed from 1 to 0
Missing values in 'City' changed from 3 to 0
Missing values in 'Marks' changed from 4 to 0
